## ETL Process of several data types

In [3]:
import glob 
import pandas as pd 
import xml.etree.ElementTree as ET 
from datetime import datetime 
  
log_file = "etl_log.txt" 
target_file = "cooked_data.csv" 
  
def extract_from_csv(file_to_process): 
    dataframe = pd.read_csv(file_to_process) 
    return dataframe 
  
def extract_from_json(file_to_process): 
    dataframe = pd.read_json(file_to_process, lines=True) 
    return dataframe 
  
def extract_from_xml(file_to_process): 
    dataframe = pd.DataFrame(columns=["name", "height", "weight"]) 
    tree = ET.parse(file_to_process) 
    root = tree.getroot() 
    for person in root: 
        name = person.find("name").text 
        height = float(person.find("height").text) 
        weight = float(person.find("weight").text) 
        dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name,"height":height, "weight":weight}])], ignore_index=True) 
    return dataframe 
  
def extract(): 
    extracted_data = pd.DataFrame(columns=['name','height','weight'])  
# create an empty data frame to hold extracted data 
     
    # process all csv files 
    for csvfile in glob.glob("source/*.csv"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index=True) 
         
    # process all json files 
    for jsonfile in glob.glob("source/*.json"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_json(jsonfile))], ignore_index=True) 
     
    # process all xml files 
    for xmlfile in glob.glob("source/*.xml"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_xml(xmlfile))], ignore_index=True) 
         
    return extracted_data 
  
def transform(data): 
    # Convert inches to meters and round off to two decimals 
    # 1 inch is 0.0254 meters 
    data['height'] = round(data.height * 0.0254,2) 
     
    # Convert pounds to kilograms and round off to two decimals 
    # 1 pound is 0.45359237 kilograms 
    data['weight'] = round(data.weight * 0.45359237,2) 
     
    return data 
  
def load_data(target_file, transformed_data): 
    transformed_data.to_csv(target_file)
    


In [4]:
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open(log_file,"a") as f: 
        f.write(timestamp + ',' + message + '\n') 

#Log the initialization of the ETL process 
log_progress("ETL Job Started") 
  
# Log the beginning of the Extraction process 
log_progress("Extract phase Started") 
extracted_data = extract() 
  
# Log the completion of the Extraction process 
log_progress("Extract phase Ended") 
  
# Log the beginning of the Transformation process 
log_progress("Transform phase Started") 
transformed_data = transform(extracted_data) 
print("Transformed Data") 
print(transformed_data.head())

  
# Log the completion of the Transformation process 
log_progress("Transform phase Ended") 
  
# Log the beginning of the Loading process 
log_progress("Load phase Started") 
load_data(target_file,transformed_data) 
  
# Log the completion of the Loading process 
log_progress("Load phase Ended") 
  
# Log the completion of the ETL process 
log_progress("ETL Job Ended") 

Transformed Data
    name  height  weight
0   alex    1.67   51.25
1   ajay    1.82   61.91
2  alice    1.76   69.41
3   ravi    1.73   64.56
4    joe    1.72   65.45


C:\Users\jenya\AppData\Local\Temp\ipykernel_18576\2127295418.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index=True)
C:\Users\jenya\AppData\Local\Temp\ipykernel_18576\2127295418.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name,"height":height, "weight":weight}])], ignore_index=True)
C:\Users\jenya\AppData\Local\Temp\ipykernel_18576

In [5]:
extracted_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    39 non-null     object 
 1   height  39 non-null     float64
 2   weight  39 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.0+ KB


In [7]:
extracted_data.describe()

,height,weight
count,39.000000,39.000000
mean,1.733846,58.526154
std,0.039244,6.232641
min,1.670000,50.970000
25%,1.720000,51.770000
50%,1.720000,57.810000
75%,1.760000,64.180000
max,1.820000,69.410000
